# Dictionary compilation from Για να κ̇οντούμε γρούσσα νάμου

This notebook aims to create a dictionary from the words contained in Ioannis Kambysis' book, _Για να κχοντούμε γρούσσα νάμου_ (2020) using Google Lens OCR.

# Preparation

In [1]:
# Data wrangling
import pandas as pd
import numpy as np

# Others
import re
import pyperclip as pc
import unicodedata
import copy
import os

## Auxiliary functions

In [2]:
def remove_accents(input_text):
    nfkd_form = unicodedata.normalize('NFKD', input_text)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [19]:
def compute_masculine_paradigm(entry_: pd.Series):
    """
    Computes the hypothesized paradigm for a given masculine entry.
    """

    # Remove accents from the  plural
    entry_['plural'] = remove_accents(entry_['plural'])

    ### Apply rules ###
    # A1: plural in -οι
    if entry_['plural'].endswith('οι'):
        return "Α1"
    
    # A4: plural in -νε 
    # NOTE: plurals are formed by adding -ούνε to the singular
    # but it is checked here as -νε for simplicity
    elif entry_['plural'].endswith('ουνε'):
        return "Α4"
    
    # A5: plural in -δε
    elif entry_['plural'].endswith('δε'):
        return "Α5"
    
    # A2: plural in -ε
    elif entry_['plural'].endswith('ε'):
        return "Α2"
    
    # A3: plural in -ου
    elif entry_['plural'].endswith('ου'):
        return "Α3"
    
    # NaN if no rule applies
    else:
        return "Α"

In [20]:
def compute_femenine_paradigm(entry_: pd.Series):
    """
    Computes the hypothesized paradigm for a given feminine entry.
    """

    # Remove accents from the word
    entry_['singular'] = remove_accents(entry_['singular'])

    ### Apply rules ###
    # Θ3: plural in -άε
    # If it does not hold, remove the accent from the plural suffix
    if entry_['plural'].endswith('άε'):
        return "Θ3"
    else:
        entry_['plural'] = remove_accents(entry_['plural'])

    # Θ2: plural in -λε
    if entry_['plural'].endswith('λε'):
        return "Θ2"    
    
    # Θ4: plural in -δε and word does not end in -ου
    elif entry_['plural'].endswith('δε') and not entry_['singular'].endswith('ου'):
        return "Θ4"

    # Θ5: plural in -δε and word ends in -ου
    elif entry_['plural'].endswith('δε') and entry_['singular'].endswith('ου'):
        return "Θ5"
    
    # Θ1: rest of plurals in ending in -ε
    elif entry_['plural'].endswith('ε'):
        return "Θ1"
        
    # NaN if no rule applies
    else:
        return "Θ"

In [21]:
def compute_neuter_paradigm(entry_):
    """
    Computes the hypothesized paradigm for a given neuter entry.
    """

    # Remove accents from the word and plural
    for key in ['singular', 'plural']:
        if entry_[key] is not np.nan:
            entry_[key] = remove_accents(entry_[key])    

    ### Apply rules ###
    # Υ2: word ends in -μα
    if entry_['singular'].endswith('μα'):
        return "Υ2"
    
    # Υ3: plural ends in -ια
    elif entry_['plural'].endswith('ια'):
        return "Υ3"
    
    # Υ4: word ends in -ι and plural ends in -τα
    elif entry_['singular'].endswith('ι') and entry_['plural'].endswith('ητα'):
        return "Υ4"
    
    # Υ5: word ends in -ε and plural ends in -τα
    elif entry_['singular'].endswith('ε') and entry_['plural'].endswith('ατα'):
        return "Υ5"
    
    # Y1: rest of plurals in -α
    elif entry_['plural'].endswith('α'):
        return "Υ1"
    
    # Υ if no rule applies
    else:
        return 'Υ'

In [22]:
def compute_paradigms(entry: pd.Series):
    """
    Extracts noun paradigms based on the plural suffixes.
    """

    # If the plural key is NaN, return the gender
    if entry['plural'] is np.nan:
        return entry['gender']

    # Create a modifyable copy of the df_dict
    # The values will lose the accents, which is not desired
    # for the final dataframe
    copy_entry = copy.deepcopy(entry)

    # Masculine nouns
    if entry['article'] == 'Ο':
        paradigm = compute_masculine_paradigm(copy_entry)
    
    # Femenine nouns
    elif entry['article'] == 'Α':
        paradigm = compute_femenine_paradigm(copy_entry)

    # Neuter nouns
    elif entry['article'] == 'Το':
        paradigm = compute_neuter_paradigm(copy_entry)

    # NaN if no rule applies
    else:
        paradigm = np.nan

    return paradigm

# Set clean text

In [8]:
page = 39

In [9]:
text = """
Ο χιούρε - Οι χιούροι ; Το γουρούνι - Τα γουρούνια 
Ο π̇ογκικό - Οι πογκιτσοί ; Ο ποντικός - Οι ποντικοί
Ο λιούκο - Οι λιούτσοι ; Ο λύκος - Οι λύκοι
Ο φορτακά - Οι φορτακάδε ; Ο βάτραχος - Οι βάτραχοι 
Ο π̇έλεχα - Οι πελέχοι ; Ο αρουραίος - Οι αρουραίοι
Ο αγό - Οι αγοί ; Ο λαγός - Οι λαγοί 
Ο ούθι - Οι ουθίουνε ; Το φίδι - Τα φίδια
Α έαφο - Οι εάφου ; Το ελάφι - Τα ελάφια
Ο άσγο - Οι άσγοι ; Ο ασβός - Οι ασβοί
Ο βρούχο - Οι βρούχοι ; Η ακρίδα - Οι ακρίδες
Ο κ̇ομπίο - Οι κομπίουνε ; Η αράχνη - Οι αράχνες
Ο κοτσ̌οφό - Οι κοτσοφοί ; Το κοτσύφι - Τα κοτσύφια 
Α π̇έντζικα - Οι πέντζιτσε ; Οι πέρδικα - Οι πέρδικες
Ο αητέ - Οι αητοί ; Ο αετός - Οι αετοί
Α μούζα - Οι μούζε ; Η μύγα - Οι μύγες
Το λιγκόνι - Τα λιγκόνια ; Το μυρμήγκι - Τα μυρμήγκια
Α τσ̌ινά - Οι τσινάε ; Η σφήκα - Οι σφήκες
Α μελισσά - Οι μελισσάε ; Η μέλισσα - Οι μέλισσες
Ο όζακα - Οι οζάκου ; Είδος μυρμηγκιού
Ο κ̇ώακα - Οι κωάκου ; Το σκουλήκι - Τα σκουλήκια
Ο κ̇όκηκα ; Ο σκόρος
Ο κοντοσέρκη ; Η οχιά
"""

In [10]:
# Save cleaned text
with open(rf'./cleaned_pages/page_{page}.txt', 'w', encoding='utf-8') as f:
    f.write(text.strip())

In [11]:
# Separate Tsakonian and Greek parts
text_list = text.strip().split('\n')
tsakonian_text = [line.split(' ; ')[0] for line in text_list]
greek_text = [line.split(' ; ')[1] for line in text_list]

assert len(tsakonian_text) == len(greek_text), 'Tsakonian and Greek parts must have the same length.'

In [12]:
# Build initial dataframe
temp_df = pd.DataFrame({'tsakonian' : tsakonian_text, 
                        'greek' : greek_text})
temp_df

,tsakonian,greek
0,Ο χιούρε - Οι χιούροι,Το γουρούνι - Τα γουρούνια
1,Ο π̇ογκικό - Οι πογκιτσοί,Ο ποντικός - Οι ποντικοί
2,Ο λιούκο - Οι λιούτσοι,Ο λύκος - Οι λύκοι
3,Ο φορτακά - Οι φορτακάδε,Ο βάτραχος - Οι βάτραχοι
4,Ο π̇έλεχα - Οι πελέχοι,Ο αρουραίος - Οι αρουραίοι
5,Ο αγό - Οι αγοί,Ο λαγός - Οι λαγοί
6,Ο ούθι - Οι ουθίουνε,Το φίδι - Τα φίδια
7,Α έαφο - Οι εάφου,Το ελάφι - Τα ελάφια
8,Ο άσγο - Οι άσγοι,Ο ασβός - Οι ασβοί
9,Ο βρούχο - Οι βρούχοι,Η ακρίδα - Οι ακρίδες


## Process Tsakonian column

### Determine gender

In [13]:
# Extract the gender
gender_dict = {
    "Ο" : "Α",
    "Α" : "Θ",
    "To" : "Υ",
    "Το" : "Υ",
    "Τα" : "Υ",
    "Οι" : np.nan,
}

# Create columns
temp_df['article'] = temp_df['tsakonian'].apply(lambda x: x.split(' ')[0])
temp_df['gender'] = temp_df['article'].apply(lambda x: gender_dict[x] if x in gender_dict.keys() else np.nan)
temp_df

,tsakonian,greek,article,gender
0,Ο χιούρε - Οι χιούροι,Το γουρούνι - Τα γουρούνια,Ο,Α
1,Ο π̇ογκικό - Οι πογκιτσοί,Ο ποντικός - Οι ποντικοί,Ο,Α
2,Ο λιούκο - Οι λιούτσοι,Ο λύκος - Οι λύκοι,Ο,Α
3,Ο φορτακά - Οι φορτακάδε,Ο βάτραχος - Οι βάτραχοι,Ο,Α
4,Ο π̇έλεχα - Οι πελέχοι,Ο αρουραίος - Οι αρουραίοι,Ο,Α
5,Ο αγό - Οι αγοί,Ο λαγός - Οι λαγοί,Ο,Α
6,Ο ούθι - Οι ουθίουνε,Το φίδι - Τα φίδια,Ο,Α
7,Α έαφο - Οι εάφου,Το ελάφι - Τα ελάφια,Α,Θ
8,Ο άσγο - Οι άσγοι,Ο ασβός - Οι ασβοί,Ο,Α
9,Ο βρούχο - Οι βρούχοι,Η ακρίδα - Οι ακρίδες,Ο,Α


### Determine plural paradigm

In [14]:
# Extract singular and plurals
temp_df['singular'] = temp_df['tsakonian'].apply(lambda x: x.split(' - ')[0].split(' ')[1])
temp_df['plural'] = temp_df['tsakonian'].apply(lambda x: x.split(' - ')[1].split(' ')[1] if ' - ' in x else np.nan)

temp_df

,tsakonian,greek,article,gender,singular,plural
0,Ο χιούρε - Οι χιούροι,Το γουρούνι - Τα γουρούνια,Ο,Α,χιούρε,χιούροι
1,Ο π̇ογκικό - Οι πογκιτσοί,Ο ποντικός - Οι ποντικοί,Ο,Α,π̇ογκικό,πογκιτσοί
2,Ο λιούκο - Οι λιούτσοι,Ο λύκος - Οι λύκοι,Ο,Α,λιούκο,λιούτσοι
3,Ο φορτακά - Οι φορτακάδε,Ο βάτραχος - Οι βάτραχοι,Ο,Α,φορτακά,φορτακάδε
4,Ο π̇έλεχα - Οι πελέχοι,Ο αρουραίος - Οι αρουραίοι,Ο,Α,π̇έλεχα,πελέχοι
5,Ο αγό - Οι αγοί,Ο λαγός - Οι λαγοί,Ο,Α,αγό,αγοί
6,Ο ούθι - Οι ουθίουνε,Το φίδι - Τα φίδια,Ο,Α,ούθι,ουθίουνε
7,Α έαφο - Οι εάφου,Το ελάφι - Τα ελάφια,Α,Θ,έαφο,εάφου
8,Ο άσγο - Οι άσγοι,Ο ασβός - Οι ασβοί,Ο,Α,άσγο,άσγοι
9,Ο βρούχο - Οι βρούχοι,Η ακρίδα - Οι ακρίδες,Ο,Α,βρούχο,βρούχοι


In [23]:
# Compute paradigms
temp_df['paradigm'] = temp_df.apply(compute_paradigms, axis=1)
temp_df

,tsakonian,greek,article,gender,singular,plural,paradigm
0,Ο χιούρε - Οι χιούροι,Το γουρούνι - Τα γουρούνια,Ο,Α,χιούρε,χιούροι,Α1
1,Ο π̇ογκικό - Οι πογκιτσοί,Ο ποντικός - Οι ποντικοί,Ο,Α,π̇ογκικό,πογκιτσοί,Α1
2,Ο λιούκο - Οι λιούτσοι,Ο λύκος - Οι λύκοι,Ο,Α,λιούκο,λιούτσοι,Α1
3,Ο φορτακά - Οι φορτακάδε,Ο βάτραχος - Οι βάτραχοι,Ο,Α,φορτακά,φορτακάδε,Α5
4,Ο π̇έλεχα - Οι πελέχοι,Ο αρουραίος - Οι αρουραίοι,Ο,Α,π̇έλεχα,πελέχοι,Α1
5,Ο αγό - Οι αγοί,Ο λαγός - Οι λαγοί,Ο,Α,αγό,αγοί,Α1
6,Ο ούθι - Οι ουθίουνε,Το φίδι - Τα φίδια,Ο,Α,ούθι,ουθίουνε,Α4
7,Α έαφο - Οι εάφου,Το ελάφι - Τα ελάφια,Α,Θ,έαφο,εάφου,Θ
8,Ο άσγο - Οι άσγοι,Ο ασβός - Οι ασβοί,Ο,Α,άσγο,άσγοι,Α1
9,Ο βρούχο - Οι βρούχοι,Η ακρίδα - Οι ακρίδες,Ο,Α,βρούχο,βρούχοι,Α1


## Process Greek column

In [24]:
# Keep only the singular form of the word for Greek
temp_df['greek'] = temp_df['greek'].apply(lambda x: x.split(' - ')[0])

# If the word starts with an article, remove it
articles = ['Ο ', 'Η ', 'Το ', 'Οι ', 'Τα ']
temp_df['greek'] = temp_df['greek'].apply(lambda x: ' '.join(x.split(' ')[1:]) if x.startswith(tuple(articles)) else x)

# Lowercase
temp_df['greek'] = temp_df['greek'].apply(lambda x: x.lower())

temp_df

,tsakonian,greek,article,gender,singular,plural,paradigm
0,Ο χιούρε - Οι χιούροι,γουρούνι,Ο,Α,χιούρε,χιούροι,Α1
1,Ο π̇ογκικό - Οι πογκιτσοί,ποντικός,Ο,Α,π̇ογκικό,πογκιτσοί,Α1
2,Ο λιούκο - Οι λιούτσοι,λύκος,Ο,Α,λιούκο,λιούτσοι,Α1
3,Ο φορτακά - Οι φορτακάδε,βάτραχος,Ο,Α,φορτακά,φορτακάδε,Α5
4,Ο π̇έλεχα - Οι πελέχοι,αρουραίος,Ο,Α,π̇έλεχα,πελέχοι,Α1
5,Ο αγό - Οι αγοί,λαγός,Ο,Α,αγό,αγοί,Α1
6,Ο ούθι - Οι ουθίουνε,φίδι,Ο,Α,ούθι,ουθίουνε,Α4
7,Α έαφο - Οι εάφου,ελάφι,Α,Θ,έαφο,εάφου,Θ
8,Ο άσγο - Οι άσγοι,ασβός,Ο,Α,άσγο,άσγοι,Α1
9,Ο βρούχο - Οι βρούχοι,ακρίδα,Ο,Α,βρούχο,βρούχοι,Α1


## Final processing

In [25]:
# Extract final table
columns = ['singular', 'greek', 'paradigm']
final_table = temp_df[columns].copy()

# Add a 'source' column with value 1
final_table['source'] = 1

final_table

,singular,greek,paradigm,source
0,χιούρε,γουρούνι,Α1,1
1,π̇ογκικό,ποντικός,Α1,1
2,λιούκο,λύκος,Α1,1
3,φορτακά,βάτραχος,Α5,1
4,π̇έλεχα,αρουραίος,Α1,1
5,αγό,λαγός,Α1,1
6,ούθι,φίδι,Α4,1
7,έαφο,ελάφι,Θ,1
8,άσγο,ασβός,Α1,1
9,βρούχο,ακρίδα,Α1,1


In [27]:
# Save the dataframe to an excel
path_ = f'./results/{page}.xlsx'
final_table.to_excel(path_, index=False)
print(f'{page} saved.')

# Open the excel
complete_path = os.path.abspath(path_)
os.startfile(complete_path)

39 saved.
